# Závěrečná zpráva k semestrální práci předmětu MI-PDP

**Marián Hlaváč**, hlavam30  
[marian.hlavac@fit.cvut.cz](mailto:marian.hlavac@fit.cvut.cz)  
27\. dubna 2018, MI-PDP, letní semestr 2017/18 

https://github.com/mmajko/mi-pdp-jes

## Problém a zadání úlohy

Ke zpracování byla předložena úloha **Jezdec na šachovnici**.

Na počátku je na čtvercové šachovnici S rozmístěno q figurek a 1 jezdec (kůň). Tomuto rozmístění figurek budeme říkat počáteční konfigurace. Jeden tah je skok jezdce podle šachových pravidel. Cílem hry je odstranit jezdcem všechny figurky s minimálním počtem tahů tak, aby na šachovnici zůstal samotný jezdec.

### Vstupní data

k = přirozené číslo, k>5, reprezentující délku strany šachovnice S o velikosti kxk  
q = přirozené číslo q<k2-1 reprezentující počet rozmístěných figurek na šachovnici S  
F[1..q] = pole souřadnic rozmístěných figurek na šachovnici S  
J = souřadnice jezdce na šachovnici S  

### Výstup algoritmu

Minimální posloupnost tahů jezdce vedoucí do cíle (tj. vedoucí do stavu, kdy jezdec zůstává sám na šachovnici). Výstupem bude seznam souřadnic políček, kam jezdec táhne s označením hvězdičkou těch políček, kde došlo k odstranění figurky.

## Použití programu

Instrukce ke spuštění programu se nacházejí v README souboru repozitáře, který lze nalézt na tomto odkaze:

https://github.com/mmajko/mi-pdp-jes/blob/master/README.md

README obsahuje instrukce, jak kód zkompilovat a jak spustit některé varianty výpočtu. Všechny varianty nejsou dostupné, jelikož poslední verze se soustředí na dostupnost výpočtů pro finální měření. Všechny verze programu, které byly použity pro odevzdávání v průběhu semestru lze najít na jednotlivých větvích repozitáře (větve `task-paralelism`, `data-paralelism` a `mpi`), pouze k nim není dostupný README soubor.

## Postup řešení

### Použité nástroje

Program je napsán v jazyce *C++* a využívá knihoven *OpenMP* a *MPI*. Je verzován systémem *Git*, odkaz na repozitář lze najít v hlavičce zprávy.

Zpráva byla vytvořena v nástroji *Jupyter*, a její notebook je napsán v jazyce *Python*.

## Sekvenční algoritmus

Algoritmus je typu BB-DFS, iterativní. Herní plocha je načtena ze souboru a program ji interně drží jako instanci třídy `Game`. Tato instance je až na výjimky immutabilní, samotné řešení se provádí nad jinou třídou.

Řešení řeší třída `Solver`, která může přijmout v konstruktoru instanci třídy `Game`, ze které se sestaví a připraví počáteční stav programu k řešení problému.

Interně si program drží koordináty na herní ploše jako jedno číslo typu `unsigned short` (předpokládal jsem, že se nebudou počítat řešení s herní plochou větší, než 256x256 políček). Samotná herní plocha ve třídě `Game` je držena jako pole těchto čísel.

Před započnutím řešení se spustí časovač pro měření doby trvání řešení úlohy. K tomuto účelu používám knihovu `std::chrono` ze standartní knihovny. Poskytuje dostatečně přesné zachycení časových momentů.

Třída `Solver` dále pracuje s instancemi třídy `Solution`, které představují určité řešení problému, ať už platné řešení, nebo neplatné. Celý algoritmus tak pracuje s prvním neplatným řešením, které obsahuje prázdnou cestu (pouze počáteční bod) a toto neplatné řešení pak vylepšuje (především hledá platné řešení).

Třída `Solution` drží cestu řešení jako vektor koordinát, tedy typu `std::vector<unsigned short>`.

Algoritmus neuvažuje žádnou lehkou heuristiku, ve smyslu "kterým směrem se vydat". Výpočet a následování této hodnoty se zdálo být stejně náročné, jako celý tento proces vynechat. 

Výstupem algoritmu je CSV soubor s napočítanými řešení všech vstupních souborů a statistiky průběhu řešení. S výsledky tak jde velmi snadno pracovat v *Jupyteru*.

## Paralelní algoritmus OpenMP

S pomocí knihovny *OpenMP* byl program jednoduše obohacen o paralelní vykonávání kódu. Byly vyzkoušeny dvě varianty paralelizace - **taskový paralelismus** a **datový paralelismus**, přičemž si datový paralelismus v mnoha případech vedl lépe (až na příliš jednoduché úlohy, tak vynikal sekvenční algoritmus pro svou jednoduchost).

*OpenMP* řeší paralelizaci, sdílení paměti a všechny rutinní činnosti okolo procesu paralelizace. Kód tak stačilo jen opatřit o `#pragma` direktivy.

Konkrétně bylo využito direktiv `#pragma omp parallel for` a `#pragma omp critical`, které plně dostačovaly k paralelizaci programu.

Sekvenční algoritmus byl iterativní a byl tak připraven pro jednoduchou paralelizaci.

Pro datový paralelismus existuje v programu podpůrná funkce `generate_queue()`, která zajistila předpočet stavů a vytvoření fronty, umožňující snadnou distribuci práce mezi jednotlivá jádra (napsána obecně tak, aby mohla být použita i pro MPI paralelizaci, více v části popisující MPI).

Počet těchto předgenerovaných stavů ovlivňoval efektivitu výpočtu, jelikož přímo ovlivňoval granularitu distribuce částí problémů na jádra.  
[Hledání nejlepšího rozdělení](https://github.com/mmajko/mi-pdp-jes/blob/data-paralelism/docs/run-local-01.txt) ukázalo, že hodnota rovna přibližně desetinásobku jader, na které bude výpočet rozdistribuován, je blízko ideální hodnotě.

## Paralelní algoritmus MPI

Knihovna *MPI* nabídla rozhraní pro komunikaci po síti. Díky tomu bylo možné rozložit paralelizaci nejen přes více vlákem jednoho procesoru, ale spustit program synchronizovaně na více výpočetních uzlech.

Algoritmus je typu master-slave. Binárka je pro všechny typy uzlů stejná, pomocí knihovny *MPI* se až uvnitř programu rozhoduje, jakou roli daná instance běhu programu přebírá.

Master algoritmus po spuštění postupně načítá soubory k výpočtu, rozešle slave uzlům broadcast informaci o tom, jaký soubor s daty se nyní počítá, rozdělí problém a několik podproblémů stejnou funkcí, jako při datové paralelizaci a tyto podproblémy rozdistribuuje do sítě. Poté master algoritmus jen čeká na vyhotovenou práci od slave uzlů, kterou dále zpracovává (porovnává, kontroluje jejich správnost apod.)

Slave algoritmus čeká na instrukce od master uzlu. Také načítá soubory a spustí výpočet. Po dokončení odešle řešení a čeká na další instrukce.

**Celá komunikace je orientována, aby byla co nejvíce minimální**, slave uzly načítají soubory samostatně a vyměňují si pouze serializované verze instancí třídy `Solution`. Přenáší se tak pouze o pole typu `SHORT`, o velikosti `n+2`, kde n je délka daného řešení, v obou směrech. Poslední dva čísla dourčují důležité vlastnosti přenášeného řešení, ale **žádná další redundantní data nejsou přenášena**. Komunikace master -> slave je zpravidla velmi malá, jelikož základní rozdělení na podproblémy vytvoří nevalidní řešení s cestou o délce zpravidla 2-4, po síti se tak přenese jako obsah pouze 4-6 čísel v poli, jako instrukce pro slave, což je max. 12 bajtů. Komunikace je díky tomu velmi rychlá a spolehlivá.

## Měření

> TODO: Uvést tabulku sekvenčních výsledku

### Data

Data, na kterých se bude provádět měření vycházejí z ukázkových dat.

## Výsledky měření

## Závěr

## Zdroje

- https://edux.fit.cvut.cz/courses/MI-PDP.16/labs/zadani_semestralnich_praci

